In [1]:
import os
import shutil
import json
from torchxrayvision.models import model_urls, get_weights

import huggingface_hub as hub

In [2]:
hub.notebook_login()

Login successful
Your token has been saved to /home/kamal_raj/.huggingface/token


In [3]:
# list of model names to upload to the hub
model_names = [
    "densenet121-res224-all",
    "densenet121-res224-nih",
    "densenet121-res224-pc",
    "densenet121-res224-chex",
    "densenet121-res224-rsna",
    "densenet121-res224-mimic_nb",
    "densenet121-res224-mimic_ch",
    "resnet50-res512-all",
]

In [4]:
for name in model_names:
    print(model_urls[name].keys())
    break

dict_keys(['description', 'weights_url', 'labels', 'op_threshs', 'ppv80_thres'])


In [5]:
# create folder for storing all model repo
local_model_dir = "hf_models"
os.makedirs(local_model_dir, exist_ok=True)

In [6]:
def add_and_commit_to_hub(repo, filepath, commit_msg):
    """
    repo: hub.Repository
    filepath : file path to add file
    commit_msg: commit message for the file
    """
    repo.git_add(os.path.abspath(filepath))
    repo.git_commit(commit_message=commit_msg)

In [7]:
def get_model_card(model_name, description, repo_name):
    # adapted from https://huggingface.co/microsoft/resnet-34/raw/main/README.md
    metadata = f"""
---
license: apache-2.0
tags:
- vision
- image-classification
datasets:
- nih-pc-chex-mimic_ch-google-openi-rsna
---
    
    """
    model_type = "resnet" if model_name.startswith("resnet") else "densenet"
    model_name = f"# {model_name}"
    if model_type == "resnet":
        model_info = """
ResNet (Residual Network) is a convolutional neural network that democratized the concepts of residual learning and skip connections. This enables to train much deeper models.
        """
    else:
        model_info = """
A DenseNet is a type of convolutional neural network that utilises dense connections between layers, through Dense Blocks, where we connect all layers (with matching feature-map sizes) directly with each other. To preserve the feed-forward nature, each layer obtains additional inputs from all preceding layers and passes on its own feature-maps to all subsequent layers.
        """
    if description is not None:
        model_info = "\n".join([model_info, description])

    how_to_use = f"""
### How to use

Here is how to use this model to classify an image of xray:

```python
#!/usr/bin/env python
# coding: utf-8

import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import argparse
import skimage, skimage.io
import pprint

import torch
import torch.nn.functional as F
import torchvision, torchvision.transforms

import torchxrayvision as xrv

parser = argparse.ArgumentParser()
parser.add_argument('-f', type=str, default="", help='')
parser.add_argument('img_path', type=str)
parser.add_argument('-weights', type=str,default="{repo_name}")
parser.add_argument('-feats', default=False, help='', action='store_true')
parser.add_argument('-cuda', default=False, help='', action='store_true')
parser.add_argument('-resize', default=False, help='', action='store_true')

cfg = parser.parse_args()


img = skimage.io.imread(cfg.img_path)
img = xrv.datasets.normalize(img, 255)  

# Check that images are 2D arrays
if len(img.shape) > 2:
    img = img[:, :, 0]
if len(img.shape) < 2:
    print("error, dimension lower than 2 for image")

# Add color channel
img = img[None, :, :]


# the models will resize the input to the correct size so this is optional.
if cfg.resize:
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                xrv.datasets.XRayResizer(224)])
else:
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop()])

img = transform(img)


model = xrv.models.get_model(cfg.weights, from_hf_hub=True)

output = {{}}
with torch.no_grad():
    img = torch.from_numpy(img).unsqueeze(0)
    if cfg.cuda:
        img = img.cuda()
        model = model.cuda()
        
    if cfg.feats:
        feats = model.features(img)
        feats = F.relu(feats, inplace=True)
        feats = F.adaptive_avg_pool2d(feats, (1, 1))
        output["feats"] = list(feats.cpu().detach().numpy().reshape(-1))

    preds = model(img).cpu()
    output["preds"] = dict(zip(xrv.datasets.default_pathologies,preds[0].detach().numpy()))
    
if cfg.feats:
    print(output)
else:
    pprint.pprint(output)
    
   
```

For more code examples, we refer to the [example scripts](https://github.com/kamalkraj/torchxrayvision/blob/master/scripts).
"""

    bibtext = """
### Citation

Primary TorchXRayVision paper: [https://arxiv.org/abs/2111.00595](https://arxiv.org/abs/2111.00595)

```
Joseph Paul Cohen, Joseph D. Viviano, Paul Bertin, Paul Morrison, Parsa Torabian, Matteo Guarrera, Matthew P Lungren, Akshay Chaudhari, Rupert Brooks, Mohammad Hashir, Hadrien Bertrand
TorchXRayVision: A library of chest X-ray datasets and models. 
https://github.com/mlmed/torchxrayvision, 2020


@article{Cohen2020xrv,
author = {Cohen, Joseph Paul and Viviano, Joseph D. and Bertin, Paul and Morrison, Paul and Torabian, Parsa and Guarrera, Matteo and Lungren, Matthew P and Chaudhari, Akshay and Brooks, Rupert and Hashir, Mohammad and Bertrand, Hadrien},
journal = {https://github.com/mlmed/torchxrayvision},
title = {{TorchXRayVision: A library of chest X-ray datasets and models}},
url = {https://github.com/mlmed/torchxrayvision},
year = {2020}
arxivId = {2111.00595},
}


```
and this paper which initiated development of the library: [https://arxiv.org/abs/2002.02497](https://arxiv.org/abs/2002.02497)
```
Joseph Paul Cohen and Mohammad Hashir and Rupert Brooks and Hadrien Bertrand
On the limits of cross-domain generalization in automated X-ray prediction. 
Medical Imaging with Deep Learning 2020 (Online: https://arxiv.org/abs/2002.02497)

@inproceedings{cohen2020limits,
  title={On the limits of cross-domain generalization in automated X-ray prediction},
  author={Cohen, Joseph Paul and Hashir, Mohammad and Brooks, Rupert and Bertrand, Hadrien},
  booktitle={Medical Imaging with Deep Learning},
  year={2020},
  url={https://arxiv.org/abs/2002.02497}
}
```
    """
    model_card = "\n".join([metadata, model_name, model_info, how_to_use, bibtext])

    return model_card

In [8]:
org = "torchxrayvision"

In [9]:
# # delete if repo existing
# for name in model_names:
#     repo_name = f"{org}/{name}"
#     hub.delete_repo(repo_id=repo_name)

In [10]:
# hub
for name in model_names:
    repo_name = f"{org}/{name}"
    # create repo
    repo_url = hub.create_repo(repo_name, exist_ok=True)
    local_repo_dir = f"{local_model_dir}/{repo_name}"
    repo = hub.Repository(local_dir=local_repo_dir, clone_from=repo_url)
    repo.git_pull()
    # download weights and move to repo folder
    model_path = get_weights(name)
    local_model_path = f"{local_repo_dir}/model.pt"
    shutil.copy(model_path, local_model_path)
    add_and_commit_to_hub(repo, local_model_path, "model file")
    # write labels and other info for model to config.json
    model_config = model_urls[name]
    description = model_config.pop("description", None)
    config_file_path = f"{local_repo_dir}/config.json"
    with open(config_file_path, "w") as f:
        json.dump(model_config, f, indent=2)
    add_and_commit_to_hub(repo, config_file_path, "config file")
    # Create model card
    model_card_file_path = f"{local_repo_dir}/README.md"
    with open(model_card_file_path, "w") as f:
        model_card = get_model_card(name, description, repo_name)
        f.write(model_card)
    add_and_commit_to_hub(repo, model_card_file_path, "model card")
    repo.git_push()

Cloning https://huggingface.co/torchxrayvision/densenet121-res224-all into local empty directory.
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file model.pt:   0%|          | 32.0k/27.1M [00:00<?, ?B/s]

To https://huggingface.co/torchxrayvision/densenet121-res224-all
   90ee2d6..746c25f  main -> main

Cloning https://huggingface.co/torchxrayvision/densenet121-res224-nih into local empty directory.
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file model.pt:   0%|          | 32.0k/27.1M [00:00<?, ?B/s]

To https://huggingface.co/torchxrayvision/densenet121-res224-nih
   e76b9bd..62a4bea  main -> main

Cloning https://huggingface.co/torchxrayvision/densenet121-res224-pc into local empty directory.
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


...

To https://huggingface.co/torchxrayvision/densenet121-res224-mimic_nb
   cea5e99..0164ff6  main -> main

Cloning https://huggingface.co/torchxrayvision/densenet121-res224-mimic_ch into local empty directory.
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file model.pt:   0%|          | 32.0k/27.1M [00:00<?, ?B/s]

To https://huggingface.co/torchxrayvision/densenet121-res224-mimic_ch
   62a3072..b876e76  main -> main

Cloning https://huggingface.co/torchxrayvision/resnet50-res512-all into local empty directory.
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file model.pt:   0%|          | 32.0k/90.1M [00:00<?, ?B/s]

To https://huggingface.co/torchxrayvision/resnet50-res512-all
   f532333..911c7ee  main -> main

